--------------------------
# <center>TP4 : Kafka et NIFI<center>
---------------------

Pas besoin de VM pour ce cours, on va travailler directement depuis vos poste telecom. Le
TP a été testé depuis une VM debian.

## Exercice 1 : Kafka

Kafka est un Message Oriented Middleware. Nous allons l’installer et le tester. Nous testerons les api entrantes et sortantes, mais aussi la robustesse de Kafka, en killant certains brokers Kafka pendant les écritures ou la lecture de messages.

### a. Installation

Dans un premier temps, nous allons procéder à l’installation d’un broker Kafka pour comprendre les notions de consumer et producer. Ensuite nous déploierons un cluster de 3 brokers Kafka sur lesquels nous testerons la robustesse.


Pour rappel, nous aurons besoin d’un service Zookeeper pour utiliser le service Kafka. Nous pourrions utiliser le zookeeper présent sur les VMs cloudera. pour des raisons de performance, je conseille d’utiliser directement les machines centos de l’école sur lesquelles nous lancerons Zookeeper et Kafka.

* Créer un dossier “kafka” sur le filesystem Linux : 
```shell
mkdir kafka
```
* Récupérer le package : 
```shell
wget https://www-eu.apache.org/dist/kafka/2.3.0/kafka_2.12-2.3.0.tgz```
* Décompresser le package : 
```shell 
tar -xzf kafka_2.11-2.0.0.tgz```
* Entrer dans le répertoire : 
```shell
cd kafka_2.11-2.0.0```
* Démarrez le service Zookeeper embarqué : 
```shell
bin/zookeeper-server-start.sh config/zookeeper.properties```


Vous pouvez vérifier les Znodes présents sur Zookeeper avec la cli (**ouvrez un autre terminal**) :
* On lance le service "client" :
```shell
bin/zookeeper-shell.sh localhost:2181```
* On arrive sur une invite de commande qui nous permet de voir différentes choses, notamment :
    + les brokers (Znodes) créés avec leurs ids : 
    ```shell
    ls /brokers/ids```
    + les topic créés : 
    ```shell
    ls /brokers/topics```
  

Une fois que Zookeeper a démarré, **ouvrez un autre terminal** et lancez le serveur Kafka :   
```shell
bin/kafka-server-start.sh config/server.properties
```


Notez que cette commande prend en entrée un fichier .properties détaillant la configuration du broker Kafka qui va être lancé. Notez la propriété broker.id, permettant d’identifier les brokers.

Parmis les options intéressantes, notez aussi le nombre de partitions par topic par défaut, la durée de rétention des messages dans Kafka par défaut. Ces configurations sont spécifiques aux topics.


Vérifiez à nouveau sur *Zookeeper cli* les Znodes créés :    
```shell
ls /brokers/ids
```



### b. Création de topic

Le service devrait maintenant tourner. Testez le en créant un premier topic Kafka :   
```shell
bin/kafka-topics.sh --create --zookeeper localhost:2181 --replication-factor 1 --partitions 1 --topic fr.telecom.tpkafka.kafkatopictest
```


Le message ***“Created topic "fr.telecom.tpkafka.kafkatopictest".”*** devrait s’afficher dans les logs du broker. Vous pouvez aussi utiliser la commande ci-dessous pour lister les topics déclarés sur votre cluster Kafka :  
```shell
bin/kafka-topics.sh --list --zookeeper localhost:2181
```


Vous retrouvez également ces informations via la *cli Zookeeper*, dans les nouveaux Znodes créés:  
```shell
ls /brokers/topics
```

### c. Producer Kafka

A présent, testez le producer shell de kafka pour écrire des messages dans le topic :
```shell
bin/kafka-console-producer.sh --broker-list localhost:9092 --topic fr.telecom.tpkafka.kafkatopictest
```
Cette commande vous ouvre une session d’écriture vers votre topic Kafka. Ecrivez un premier message, faites entrée pour envoyer le message au topic.


### d. Consumer Kafka

**Dans un autre terminal**, lancez un consumer via :
```shell
bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic fr.telecom.tpkafka.kafkatopictest --from-beginning`
```
Vous devriez voir apparaître les messages que vous avez envoyé depuis votre producer.   
Votre consumer possède un offset sur le topic Kafka qui vous permet de récupérer les derniers messages non lus. Cependant, l’option `--from-beginning` vous permet de reparcourir l’ensemble des données disponibles dans le topic, sans se soucier de l’offset.     
Ces deux commandes possèdent plusieurs options que vous pouvez afficher en les lançant sans arguments.


Tapez quelques messages dans votre producer et regardez les apparaitres dans le consumer.


### e. Un cluster multi-broker

Pour l’instant nous n’avons utilisé qu’un seul broker. Pour tester le parallélisme et la haute tolérance aux pannes de Kafka nous allons créer un cluster de 3 brokers.      

Un broker est définit par un fichier de configuration, dans lequel est spécifié l’id du broker, le port sur lequel le broker écoute, et le dossier dans lequel les données seront écrites.     
Afin de spécifier les deux autres brokers, créez deux nouveaux fichiers properties en partant du fichier *server.properties*.
```shell
cd ~/kafka # bien se placer dans le fichier d'installation de kafka
cp config/server.properties config/server-1.properties
cp config/server.properties config/server-2.properties
```

Dans ces deux fichiers, en trouvant les lignes correspondantes, modifiez certaines lignes afin d’avoir :
* *config/server-1.properties* :     
        broker.id=1      
        listeners=PLAINTEXT://:9093     
        log.dir=/tmp/kafka-logs-1   
* *config/server-2.properties* :    
        broker.id=2    
        listeners=PLAINTEXT://:9094   
        log.dir=/tmp/kafka-logs-2    


Un de nos brokers tourne déjà. Nous allons maintenant lancer les deux autres brokers.
```shell
bin/kafka-server-start.sh config/server-1.properties &
bin/kafka-server-start.sh config/server-2.properties &
```

Créez maintenant un nouveau topic, de 1 partition, avec un facteur de réplication de 3 :
```shell
bin/kafka-topics.sh --create --zookeeper localhost:2181 --replication-factor 3 --partitions 1 --topic fr.telecom.tpkafka.topicreplication
```

Pour voir la façon dont sont réparties les partitions et les réplicas sur un topic, tapez la commande ci-dessous :
```shell
bin/kafka-topics.sh --describe --zookeeper localhost:2181 --topic fr.telecom.tpkafka.topicreplication
``` 
On doit obtenir une réponse du genre :

        Topic:fr.telecom.tpkafka.topicreplication 
        PartitionCount:1 
        ReplicationFactor:3
        Configs:
        Topic: fr.telecom.tpkafka.topicreplication 
        Partition: 0 Leader: 2 Replicas: 1,2,0
        Isr: 1,2,0
        
Dans mon exemple, ci dessus, on voit que le leader est le broker numéro 2, et qu’il y a 3 réplicas (sur les brokers 1,2 et 0).   

**Nous allons maintenant tester la tolérance aux pannes.**

Ecrivez quelques messages dans le topic via la commande du producer :
```shell
bin/kafka-console-producer.sh --broker-list localhost:9092,localhost:9093,localhost:9094 --topic fr.telecom.tpkafka.topicreplication
```        
Consommez les avec `bin/kafka-console-consumer.sh` (*à compléter*), comme précédemment.

**Quittez le consumer**

Maintenant killez le broker sur lequel est la partition leader :
* vous pouvez quitter le terminal du broker correspondant, en fermant le terminal ou avec `Ctrl+C`
* ou utiliser ps faux | grep kafka pour retrouver l’id puis killer le process avec kill -9)

Via la commande `describe`, notez que le leader a changé. Avec le producer, envoyez quelques nouveaux messages.    
**Relancez le consumer**. Observez que les données sont toujours accessibles.      
**Relancez maintenant le broker** que vous avez éteint.

#### Créez un script bash qui envoie un message contenant la date et l’heure à la nanoseconde tout les secondes dans le topic